# Publishing packages in PyPI

One major reason for Python's popularity is its vast repository of packages.

The most common way by far to share the tools is to pack your code into a package and upload it to the Python Package Index ([PyPI](https://pypi.org)).

Once uploaded, users will be able to download your package easily using the `pip` tool.

The following section serves as a runbook describing how you can publish the library `vec2d`, which contains two separate modules (`vec2d.math` and `vec2d.graph`).

The library is maintained in a standalone Github project: https://sergiofgonzalez/vec2d

## Preparing the files

The first step is to organize the files in a particular way. Recall that the library contains two separate packages for the graphing and Math related capabilities.

```
vec2d/
    ├─── vec2d/
        ├─── graph/
            ├─── __init__.py
            └─── vector2d_graphics.py    
        └─── math/
            ├─── __init__.py
            └─── vector2d_math.py        
    ├─── .gitignore
    ├─── LICENSE
    ├─── README.md
    └─── setup.py
```

The `README.md` file contains information about the package, such as installation and usage instructions.

The `setup.py` is one of the ways in which you can provide the metadata to publish your package. This relies in the `setuptools` built-in module.

For this particular library with two separate packages looks like the following:

```python
from setuptools import setup

with open("README.md") as file:
    read_me_description = file.read()

setup(
    name="vec2d",
    version="0.1.0-rc1",
    author="Sergio F. Gonzalez",
    author_email="sergio.f.gonzalez@gmail.com",
    description="Maths and Graph functions for vectors on the 2D plane",
    long_description=read_me_description,
    long_description_content_type="text/markdown",
    url="https://github.com/sergiofgonzalez/vec2d",
    packages=["vec2d/graph", "vec2d/math"],
    include_package_data=True,
    install_requires=["matplotlib"],
    classifiers=[
        "Programming Language :: Python :: 3",
        "License :: OSI Approved :: MIT License",
        "Operating System :: OS Independent",
    ],
    python_requires=">=3.10",
)
```

The `__init__.py` in the corresponding packages serves as a marker for Python, so that the directory is taken as a Python package.

In this case, with a couple of packages, the `__init__.py` look like:

```python
"""
__init__.py for the vector2d.graph module which provides the graphic/drawing
capabilities.
"""
from vec2d.graph.vector2d_graphics import (
    Arrow,
    Colors,
    Points,
    Polygon,
    Segment,
    draw,
)

__all__ = ["Colors", "Segment", "Points", "Polygon", "Arrow", "draw"]
```

```python
"""
__init__.py for the vector2d.math module which provides the math capabilities.
"""
from vec2d.math.vector2d_math import (
    add,
    add_mult,
    displacement,
    distance,
    length,
    opposite,
    perimeter,
    rescale,
    rotate,
    scalar_product,
    subtract,
    to_cartesian,
    to_degrees,
    to_polar,
    to_radians,
    translate,
)

__all__ = [
    "add",
    "add_mult",
    "length",
    "scalar_product",
    "opposite",
    "subtract",
    "translate",
    "displacement",
    "distance",
    "perimeter",
    "to_cartesian",
    "to_radians",
    "to_degrees",
    "to_polar",
    "rotate",
    "rescale",
]
```

With those files in places, end users will be able to use those packages using:

```python
from vec2d.math import to_polar
from vec2d.graph import draw, Arrow, Colors
```


If the `__init__.py` were to be removed, Python would no longer look for submodules inside those directories, and therefore, attempts to import the module will fail.

| NOTE: |
| :---- |
| Python favors a flat structure vs. a hierarchical one ("flat is better than nested" philosophy). |

Note also that the `__init__.py` contains a list of all symbols explicitly exported by the packages:

```python
__all__ = [
  "func1",
  "func2",
  "var1",
]
```

Thus, when you do `from packname import *` all the names not prefixed by `_` would be imported.

## Registering a Test PyPI account

The first step when publishing your package consists in register a Test PyPI account. 

PyPI provides a test instance so that you can test your package, without messing up the official indexing.

In order to do that, you need to register for an account at https://test.pypi.org/account/register/.

Follow the registration steps, opt in for the 2FA and generate token to publish your package.

## Configuring your `~/.pypirc`

Next step you need to follow is configure your `~/.pypirc` file, so that it includes a reference to the Test PyPI registry and your token.

In my case it looks like the following:

```
[distutils]
index-servers = 
        pypi
        testpypi
        codeartifact

[testpypi]
  username = __token__
  password = pypi-AgEN...

[codeartifact]
repository = https://....d.codeartifact.us-east-1.amazonaws.com/pypi/prj-pypi/
username = aws
password = eyJ...
```

If not correctly configured, you'll obtain errors when uploading the package:

```
$ python3 -m twine upload --repository testpypi dist/*
ERROR    InvalidConfiguration: Missing 'testpypi' section from ~/.pypirc.                               
         More info: https://packaging.python.org/specifications/pypirc/ 
```

## Installing the publishing tools

To publish the package we need these tools, which should be installed in the virtual environment:

```bash
# Create the virtual env using conda as the base
$ conda run -n base python -m venv .venv --upgrade-deps

# Activate the environment
$ source .venv/bin/activate

# Install the required tools
(.venv) $ python -m pip install --upgrade setuptools wheel twine
```

## Packaging your library

With the tools in place, you should run the following command to create the distributable package:

```bash
# Create the archive
(.venv) $ python setup.py sdist bdist_wheel
```

This will create a few additional files that were not present before:

+ `build/`
+ `dist/`
+ `vec2d.egg-info/`

| NOTE: |
| :---- |
| If for some reason you need to recreate the archive, execute the following command to clean those dirs: `rm -rf build/ dist/ vec2d.egg-info/`. |

## Publishing in Test PyPI

To publish your package type:

```bash
# Publish in testpypi
(.venv) $ python -m twine upload --repository testpypi dist/*
```

This will take package and publish it in the Test PyPI instance:

```
Uploading distributions to https://test.pypi.org/legacy/
Uploading vec2d-0.1.0rc1-py3-none-any.whl
100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 kB • 00:00 • ?
Uploading vec2d-0.1.0rc1.tar.gz
100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 kB • 00:00 • ?

View at:
https://test.pypi.org/project/vec2d/0.1.0rc1/
```

If you point your browser to that URL you will see how your package has been published:

![Published in Test PyPI!](pics/published_test_pypi.png)

## Testing from Test PyPI

In order to test the library you can create a virtual environment and run:

```bash
(.venv) $ pip install -i https://test.pypi.org/simple/ vec2d==0.1.0rc1
```

However, if your library requires additional dependencies, you will see that those are not downloaded if they're not available in Test PyPI.

In order to solve that you can do instead:

```bash
(.venv) $ python -m pip install vec2d==0.1.0.rc1 --extra-index-url=https://test.pypi.org/simple/
```

You can then try with something like:

```python
"""
Sample program illustrating how to use the vec2d library API
"""
from vec2d.graph import draw, Segment, Points, Polygon, Arrow, Colors
from vec2d.math import add

if __name__ == "__main__":
    draw(
        Segment((0, 3), (5, 3), color=Colors.ORANGE),
        Points((0, 3), (5, 3), color=Colors.BLUE),
        Polygon((3, 1), (6, 3), (5, 1), fill=Colors.PINK),
        Arrow((5, 3), color=Colors.ORANGE),
        Arrow((3, 1), (5, 3), color=Colors.PINK),
        Arrow(add((5, 3), (3, 1)), color=Colors.BROWN)
)
```

## Publishing in the official PyPI

The server for the official PyPI is completely different from the test one, so you will need to:

1. Register yourself in https://pypi.org
2. Create a new API token
3. Configure your `~/.pypirc` with the API token

In particular, for the latter, it should look like the following:

```
$ cat ~/.pypirc
[distutils]
index-servers =
        pypi
        testpypi
        codeartifact

[pypi]
  username = __token__
  password = pypi-Ag...w

[testpypi]
  username = __token__
  password = pypi-Ag...n

[codeartifact]
...
```

With all those prerequisites in place, you just need to run the `twine upload` command, without any additional `--repository` parameters.

Note that it is recommended to re-package the library (just in case there was some version change).

```bash

$ python -m twine upload dist/*
Uploading distributions to https://upload.pypi.org/legacy/
Uploading vec2d-0.1.0-py3-none-any.whl
100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 kB • 00:00 • ?
Uploading vec2d-0.1.0.tar.gz
100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 kB • 00:00 • ?

View at:
https://pypi.org/project/vec2d/0.1.0/
```

If everything goes well, you should be able to visit the page and `pip install` your library in your projects.

| NOTE: |
| :---- |
| It is a good practice to do `git tag v0.1.0` to create a tag in sync with your published repo.<br>You can push your tag doing `git push --tags`. |